In [2]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import Xception
import tensorflow as tf

In [3]:
# Some constant
BATCH_SIZE = 32
IMG_SIZE = 224
LABEL_MODE = "categorical"

In [4]:
def process_normalize(image, label):
    #print("call process_normalize")
    image = tf.cast(image/255., tf.float32)
    # Transformation needed here
    return image, label

In [5]:
def load_image_dataset_from_directory(directory):
    ds = image_dataset_from_directory(directory,
                                      batch_size=BATCH_SIZE,
                                      image_size=(IMG_SIZE, IMG_SIZE),
                                      label_mode=LABEL_MODE)
    #print(ds.class_names)
    return ds.prefetch(int(BATCH_SIZE/2)).map(process_normalize), ds.class_names

In [7]:
train_ds, labels = load_image_dataset_from_directory("../data/preprocessed/img/Train/")
test_ds, _ = load_image_dataset_from_directory("../data/preprocessed/img/Test/")
val_ds, _ = load_image_dataset_from_directory("../data/preprocessed/img/Validation/")

NB_CLASS = len(labels)
labels

Found 2873 files belonging to 2 classes.
Found 957 files belonging to 2 classes.
Found 957 files belonging to 2 classes.


['Begnin', 'Maligne']

In [69]:
model = Xception(weights=None, input_shape=(IMG_SIZE, IMG_SIZE, 3), classes=NB_CLASS)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics="accuracy")
history = model.fit(train_ds, epochs=2, validation_data=val_ds)

Epoch 1/2
53/90 [================>.............] - ETA: 32:29 - loss: 1.6210 - accuracy: 0.6021

KeyboardInterrupt: 

In [42]:
#model.evaluate(test_ds)

30/30 [==============================] - 511s 17s/step - loss: 0.4610 - accuracy: 0.7900


[0.4609569311141968, 0.7899686694145203]

In [70]:
y_pred = model.predict(test_ds)
y_pred

array([[0.44492134, 0.5550786 ],
       [0.44487807, 0.55512196],
       [0.4448538 , 0.55514616],
       ...,
       [0.44484904, 0.555151  ],
       [0.4448799 , 0.5551201 ],
       [0.4448542 , 0.5551458 ]], dtype=float32)

In [30]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

In [71]:
y_true =  np.array([])
for _, y in test_ds:
    y_true = np.concatenate([y_true, np.argmax(y.numpy(), axis=-1)])

In [72]:
cm = confusion_matrix(y_true, np.argmax(y_pred, axis=-1))

In [74]:
cm_disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                 display_labels=["Begnin", "Malignant"])
cm_disp.plot() 

TypeError: __init__() got an unexpected keyword argument 'cmap'

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt
sn.heatmap(cm, annot=True)
plt.show()

In [44]:
from sklearn.metrics import classification_report

In [45]:
print(classification_report(y_true, np.argmax(y_pred, axis=-1), target_names=["Begnin", "Malignant"]))

              precision    recall  f1-score   support

      Begnin       0.44      0.31      0.36       454
   Malignant       0.51      0.64      0.56       503

    accuracy                           0.48       957
   macro avg       0.47      0.47      0.46       957
weighted avg       0.47      0.48      0.47       957



In [9]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import datasets
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2
from keras import optimizers

In [12]:
#!pip install -U efficientnet
from keras import layers
import efficientnet.keras as efn
from keras import regularizers 

eff_net = efn.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
model=Sequential()
model.add(eff_net)
model.add(layers.GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(NB_CLASS, activation='softmax'))

alpha = 1e-3  # weight decay coefficient
for layer in model.layers:
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.Dense):
        layer.add_loss(lambda: keras.regularizers.l2(alpha)(layer.kernel))
    if hasattr(layer, 'bias_regularizer') and layer.use_bias:
        layer.add_loss(lambda: keras.regularizers.l2(alpha)(layer.bias))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Functional) (None, 7, 7, 1280)        4049564   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               163968    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 258       
Total params: 4,214,302
Trainable params: 4,172,030
Non-trainable params: 42,272
_______________________________________

In [13]:
# callback - if accuracy doesn't improve after 3 epoch then reduce lr by factor 0.5 
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,patience=3, min_lr=0.000005)

In [21]:
from tensorflow.keras.callbacks import Callback
class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        loss, acc = self.model.evaluate(self.test_data, verbose=1)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [14]:
# compile the model 
optimizer=optimizers.Adam(lr=0.001)
model.compile(optimizer=optimizer,loss="categorical_crossentropy", metrics=['accuracy'])


ValueError: too many values to unpack (expected 2)

In [ ]:
# train the model
history = model.fit(train_ds, epochs=1, validation_data=val_ds, callbacks=[reduce_lr, TestCallback(test_ds)])

 2/90 [..............................] - ETA: 13:21 - loss: 0.0117 - accuracy: 1.0000

In [17]:
model.evaluate(test_ds)

30/30 [==============================] - 123s 4s/step - loss: 0.5541 - accuracy: 0.8777


[0.5540841221809387, 0.8777429461479187]

In [ ]:
y_pred = model.predict(test_ds)
y_pred

In [18]:
model.save("efficientnet_b0_v1")


INFO:tensorflow:Assets written to: efficientnet_b0_v1/assets
